In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 1. 数据预处理和加载
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

以上代码下载的数据集为FashinMNIST，有`10`个标签，依次为：
```python
0: T-shirt/top
1: Trouser
2: Pullover
3: Dress
4: Coat
5: Sandal
6: Shirt
7: Sneaker
8: Bag
9: Ankle boot
```


下面的代码可能运行10分钟，使用的GPU如下信息如下。

In [6]:
import torch

# 检测是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 输出当前设备是 CPU 还是 GPU
print(f'Using device: {device}')

# 如果使用 GPU，还可以查看 GPU 的信息
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))  # 输出 GPU 名称

Using device: cuda
NVIDIA GeForce RTX 3060 Laptop GPU


In [8]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # 输入层（784），隐藏层1（128），隐藏层2（64），输出层（10 类）
        self.fc1 = nn.Linear(28 * 28, 128)  # W1, b1
        self.fc2 = nn.Linear(128, 64)       # W2, b2
        self.fc3 = nn.Linear(64, 10)        # W3, b3
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 将28x28的图像拉伸为1维
        x = torch.sigmoid(self.fc1(x))  # 激活函数 Sigmoid
        x = torch.relu(self.fc2(x))  # 激活函数 ReLU
        x = self.fc3(x)  # 输出层没有激活，因为我们使用了损失函数CrossEntropyLoss
        return x

model = SimpleNet()
criterion = nn.CrossEntropyLoss()  # 使用交叉熵损失函数
optimizer = optim.SGD(model.parameters(), lr=0.01)  # 使用随机梯度下降，学习率为0.01
num_epochs = 20  # 训练轮次

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in trainloader:
        # 清除梯度
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # 反向传播
        loss.backward()
        
        # 更新参数
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}')

# 5. 测试网络
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

torch.save(model.state_dict(), 'simple_net.pth')

KeyboardInterrupt: 

In [2]:
# 创建模型实例
model = SimpleNet()
# 加载模型参数
model.load_state_dict(torch.load(r"D:\machine_learning\DeepL\model\simple_net.pth"))
model.eval()  # 设置为评估模式

NameError: name 'SimpleNet' is not defined

In [3]:
from PIL import Image
import torchvision.transforms as transforms

# 定义图片预处理
transform = transforms.Compose([
    transforms.Grayscale(),  # 转为灰度图
    transforms.Resize((28, 28)),  # 调整为28x28
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize((0.5,), (0.5,))  # 归一化
])

# 加载图片并预处理
image = Image.open(r"D:\machine_learning\DeepL\img\dress.jpg")  # 替换为你的图片路径
image = transform(image).unsqueeze(0)  # 添加批次维度

# 进行预测
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output, 1)

# 输出预测结果
print(f'Predicted class: {predicted.item()}')


NameError: name 'torch' is not defined

很显然，预测失败，训练的程度已经很好了，接下来就是超参数训练

*Created by Liu* `2024/10/15 19:30`